<a href="https://colab.research.google.com/github/jamescavanagh/unpackAI/blob/main/Tomato_Leaf_Health_Classifier_Improvements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 727kB 6.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 204kB 14.5MB/s 
     |████████████████████████████████| 1.2MB 19.2MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
Mounted at /content/gdrive
Mounted at /content/drive


In [3]:
# Orignal Data Set

root_dir = "/content/drive/MyDrive/Colab Data"
base_dir = root_dir + '/Tomato_Sample'
tomato_healthy_dir = base_dir + '/Tomato__Healthy/Tomato__Healthy'
tomato_diseased_dir = base_dir + '/Tomato__Diseased/Tomato__Diseased'
list_tomato_healthy = [os.path.join(tomato_healthy_dir, f) for f in os.listdir(tomato_healthy_dir)]
list_tomato_diseased = [os.path.join(tomato_diseased_dir, f) for f in os.listdir(tomato_diseased_dir)]

# New Data Set



The improvements that will come out the model will be done by running the model on other parts of the dataset which had more labeled examples of leaves.

At the beginning of the project, there was a problem with unbalanced datasets. Since there were around sixty thousand images in the dataset, it made sense to collect a smaller sample of a few thousand images and leave the rest alone.

However, after the first iteration of running the model, it became clear that the model needed to be trained on more examples of leaves that are only slightly diseased rather than obvious. 

Due to the impracticality of searching manually through 60,000 images, the best approach to this problem would be to run the model on the rest of the dataset to find more false positives that could be used to train the model 

So, the model was run on the rest of the data set in order to find more examples of false positives. Out of this, a similar rate of around 12 percent was found in the rest of the dataset, which showed that the data was an accurate sample of the larger set of images.

Through this, the model was run on an additional 3000 labeled images, in order to identify an additional 276 false negatives. These were then inserted in order to create a training set that had more edge examples that would be crucial to identifying the right pathway  




In [4]:
root_dir = "/content/drive/MyDrive/Colab Data"
base_dir = root_dir + '/Tomato_Sample'
tomato_sample2 = base_dir + '/'
#list_tomato_healthy = [os.path.join(tomato_healthy_dir, f) for f in os.listdir(tomato_healthy_dir)]
list_tomato_sample2 = [os.path.join(tomato_diseased_dir, f) for f in os.listdir(tomato_diseased_dir)]

In [7]:
# Trained Model Path

model_path = '/content/drive/MyDrive/Colab Data/Models/Leaves_Model.mdl'

In [11]:
# Running the model
class DataLoaders(GetAttr):
  def __init__(self, *loaders): self.loaders = loaders
  def __getitem__(self, i): return self.loaders[i]
  train,valid = add_props(lambda i, self: self[i])


In [12]:
leaves = DataBlock (
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=99),
    get_y=parent_label,
    item_tfms=Resize(256,256))

dls = leaves.dataloaders
destination =Path(base_dir)
path = destination
fns = get_image_files(path)
dls = leaves.dataloaders(path)

In [13]:
leaves = leaves.new(
    item_tfms=RandomResizedCrop(256, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = leaves.dataloaders(path)

In [35]:
modelPath = '/content/drive/MyDrive/Colab Data/Models/Leaves_Model.mdl'
#learn = load_learner(model_path,dls)
learner = load_learner('/content/drive/MyDrive/Colab Data/Models/Leaves_Model.mdl')

In [50]:
diseased_results = []
false_negatives = []

for i in list_tomato_diseased:
  result = learner.predict(i)
  diseased_results.append(result)
  if result[0] == 'Healthy':
    false_negatives.append(i)#file paths      

In [51]:
healthy_results = []
false_positives = []

for i in list_tomato_diseased:
  result = learner.predict(i)
  healthy_results.append(result)
  if result[0] == 'Diseased':
    false_positives.append(i)#file paths      

In [52]:
len(false_positives)

1508

In [53]:
len(false_negatives)

11

for this iteration it showed that the model produced many more false positives